Glycine divided into the following molecules according to Pennylane datasets:
- CO
- OH
- CH2
- H3N

In [1]:
import pennylane as qml

def parse_hamiltonian(input_string):
    terms = input_string.split('+')
    coefficients = []
    observables = []
    for term in terms:
        bracket_start = term.find('[')
        bracket_end = term.find(']')
        if bracket_start != -1 and bracket_end != -1:
            coefficient = float(term[1:bracket_start].strip().replace('(', '').replace(')', ''))  
            op = term[bracket_start + 1:bracket_end].strip()
            coefficients.append(coefficient)
            if op.startswith('I'):
                observable = qml.Identity(int(op[1:]))
            elif op.startswith('Z'):
                observable = qml.PauliZ(int(op[1:]))
            observables.append(observable)
    hamiltonian = qml.Hamiltonian(coefficients, observables)
    return hamiltonian

In [2]:
#only the first 10 coeficients ???
H3N_string = "(-26.26943471521906) [I0]+ (0.09640312931255435) [Z10]+ (0.09640312931255435) [Z11]+ (0.2090380082509059) [Z14]+ (0.20903800825090596) [Z15]+ (0.20903800893241595) [Z13]+ (0.20903800893241617) [Z12]+ (1.0617137287315201) [Z9]+ (1.0617137287315204) [Z8]+ (1.1876480803426634) [Z7]"
CO_string = "(-68.71682341693746) [I0]+ (1.0123810311207575) [Z19]+ (1.0123810311207582) [Z18]+ (1.0915901989337073) [Z12]+ (1.0915901989337078) [Z13]+ (1.0946840882385707) [Z16]+ (1.0946840882385707) [Z17]+ (1.0946840882385709) [Z14]+ (1.0946840882385713) [Z15]+ (1.4912828480575426) [Z10]"
OH_string = "(-46.737168163199584) [I0] + (0.9042993468617868) [Z10]+ (0.9042993468617868) [Z11] + (1.1738175106319706) [Z4] + (1.1738175106319708) [Z5] + (1.3519259337248126) [Z9]+ (1.3519259337248128) [Z8]+ (1.3519259337256437) [Z6]+ (1.351925933725644) [Z7]+ (1.6080522580438361) [Z3]"
CH2_string = "(-24.01803924608539) [I0]+ (0.06715870684193287) [Z13]+ (0.06715870684193298) [Z12]+ (0.11775486441472183) [Z11]+ (0.11775486441472197) [Z10]+ (0.404005256496446) [Z9]+ (0.4040052564964463) [Z8]+ (0.4152399806340434) [Z6]+ (0.4152399806340436) [Z7]+ (0.45547409692705754) [Z5]"
H3N = parse_hamiltonian(H3N_string)
CO = parse_hamiltonian(CO_string)
OH = parse_hamiltonian(OH_string)
CH2 = parse_hamiltonian(CH2_string)

In [3]:
def num_qubits_from_hamiltonian(hamiltonian):
    max_index = 0
    for term in hamiltonian.ops:
        if isinstance(term, qml.operation.Observable):
            for wire in term.wires:
                max_index = max(max_index, wire)
    num_qubits = max_index + 1
    return num_qubits

In [4]:
quH3N = num_qubits_from_hamiltonian(H3N)
quCO =  num_qubits_from_hamiltonian(CO)
quOH = num_qubits_from_hamiltonian(OH)
quCH2 = num_qubits_from_hamiltonian(CH2)

In [7]:
from pennylane import numpy as np
import optax
import jax

def ground_state_energy(H, qubits):
    dev = qml.device("lightning.qubit", wires=qubits)
    electrons = 2
    hf = qml.qchem.hf_state(electrons, qubits)
    @qml.qnode(dev)
    def circuit(param, wires):
        qml.BasisState(hf, wires=wires)
        qml.DoubleExcitation(param, wires=[0,1,2,3])
        return qml.expval(H)
    def cost_fn(param):
        return circuit(param, wires=range(qubits))
    max_iterations = 100
    conv_tol = 1e-06
    opt = optax.sgd(learning_rate=0.4)
    theta = np.array(0.)
    energy = [cost_fn(theta)]
    angle = [theta]
    opt_state = opt.init(theta)
    for n in range(max_iterations):
        gradient = jax.grad(cost_fn)(theta)
        updates, opt_state = opt.update(gradient, opt_state)
        theta = optax.apply_updates(theta, updates)
        angle.append(theta)
        energy.append(cost_fn(theta))
        conv = np.abs(energy[-1] - energy[-2])
        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
        if conv <= conv_tol:
            break
    return print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")

In [10]:
energyOH = ground_state_energy(OH, quOH)
energyH3N = ground_state_energy(H3N, quH3N)
energyCO = ground_state_energy(CO, quCO)
energyCH2 = ground_state_energy(CH2, quCH2)

Step = 0,  Energy = -35.56517792 Ha

Final value of the ground-state energy = -35.56517792 Ha
Step = 0,  Energy = -21.92940140 Ha

Final value of the ground-state energy = -21.92940140 Ha
Step = 0,  Energy = -58.63886261 Ha

Final value of the ground-state energy = -58.63886261 Ha
Step = 0,  Energy = -21.55424690 Ha

Final value of the ground-state energy = -21.55424690 Ha


In [13]:
glycine =-35.56517792-21.92940140-58.63886261-21.55424690
print(glycine)

-137.68768883
